<a href="https://colab.research.google.com/github/xiokuai/katago-ikatago/blob/main/%E5%87%A4%E9%B8%A3ikatago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

更新于北京时间2025年04月10日19:00

说明：

1.此程序提供TensorRT与CUDA引擎。

使用时可参考本人在B站发的视频：

https://www.bilibili.com/video/BV1bn6tYeE4B/?share_source=copy_web&vd_source=bad2ceb2f52b7f81a9436e832aadda15

2.使用之前先选择使用T4 GPU。如使用TensorRT后端会有额外约两分钟的配置环境时间，请耐心等待。（TensorRT）已为28b与HumanSL提供timing cache，连接即用。线程已调到最佳，无需修改。

3.连接的命令行为：

ikatago.exe -p all -u 用户名 --password 密码

注意这里用的platform不是colab。

4.配置环境后包中有28b 18b 40b 30b 20b权，可以在命令行中使用--kata-weight 指定。以使用18b为例：

ikatago.exe -p all -u 用户名 --password --kata-weight 18b

5.需要使用自己的权重就在下面一个单元格最后一行的wget后面加上权重下载链接，但权重必须以.bin.gz结尾

6.支持humansl，但只能用28b权重带。8D以下命令行为：（以8段水平为例）

ikatago.exe -p all -u 用户名 --password 密码 --kata-name humansl8d

启动引擎后可在gtp控制区输入kata-set-param humanSLProfile rank_8d，将8d改为需要的等级即可，例如20k、5k、1d、5d等

9d的命令行这样写：

ikatago.exe -p all -u 用户名 --password 密码 --kata-name humansl9d

In [ ]:
# 配置环境（务必启动这个单元格！等运行完毕再点下面的单元格）

backend = 'CUDA' # 引擎后端，可填trt（tensorrt）或cuda
backend = backend.lower()
if backend == 'trt' or backend == 'tensorrt' or backend == 'cuda':
  pass
else:
  raise ValueError('参数backend值无效，请检查输入。')

import requests
downloadURL='https://github.com/FengmingGo/katago_for_colaboratory/releases/' # 后面除了katago权重、libzip之外其他所有文件都在此目录下，Ctrl+单击可进入。

# 装libssl
%cd ~
!wget {downloadURL}download/20241228/libssl1.1_1.1.1f-1ubuntu2.17_amd64.deb
!dpkg -i libssl1.1_1.1.1f-1ubuntu2.17_amd64.deb && rm libssl1.1_1.1.1f-1ubuntu2.17_amd64.deb
# 装libzip
!wget https://libzip.org/download/libzip-1.11.2.tar.gz
!tar -zxvf libzip-1.11.2.tar.gz
%cd libzip-1.11.2
!cmake . && make && make install
!ldconfig /usr/local/lib && ldconfig /usr/local/lib64
%cd ..
!rm -rf libzip-1.11.2 && rm libzip-1.11.2.tar.gz

!wget {downloadURL}download/20250410/work.zip && unzip work
%cd ~/work/data
if backend == 'cuda': # 如使用CUDA后端
  !wget https://github.com/lightvector/KataGo/releases/download/v1.16.0/katago-v1.16.0-cuda12.8-cudnn9.8.0-linux-x64.zip && unzip katago-v1.16.0-cuda12.8-cudnn9.8.0-linux-x64 && rm katago-v1.16.0-cuda12.8-cudnn9.8.0-linux-x64.zip
else: # 如选用TensorRT后端
  !wget https://github.com/lightvector/KataGo/releases/download/v1.16.0/katago-v1.16.0-trt10.2.0-cuda12.5-linux-x64.zip && unzip katago-v1.16.0-trt10.2.0-cuda12.5-linux-x64 && rm katago-v1.16.0-trt10.2.0-cuda12.5-linux-x64.zip
  # 装TensorRT
  !apt-get install libnvinfer10=10.2.0.19-1+cuda12.5
  # 下载timing cache
  !mkdir -p /root/.katago/trtcache/
  %cd /root/.katago/trtcache/
  !wget {downloadURL}download/20250410/trt-100200_gpu-131365bf_tune-1bf250279187_exact19x19_batch12_fp16
  !wget {downloadURL}download/20250410/trt-100200_gpu-131365bf_tune-1bf250279187_exact19x19_batch8_fp16
  !wget {downloadURL}download/20250410/trt-100200_gpu-131365bf_tune-460539d595d8_exact19x19_batch8_fp16

!chmod -R 777 ~/work/
%cd ~/work/data
!rm gtp_human5k_example.cfg && rm gtp_human9d_search_example.cfg
strongest_weight = requests.get("https://katagotraining.org/api/networks/get_strongest/?format=json").text.split("\"name\":")[1].split(",\"created_at\"")[0] # 获取当前最强权重的名字
!wget https://github.com/lightvector/KataGo/releases/download/v1.15.0/b18c384nbt-humanv0.bin.gz -O humansl.bin.gz
!wget https://media.katagotraining.org/uploaded/networks/models/kata1/{strongest_weight}.bin.gz -O 28b.bin.gz
!wget https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b18c384nbt-s9996604416-d4316597426.bin.gz -O 18b.bin.gz
!wget https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s11840935168-d2898845681.bin.gz -O 40b.bin.gz
!wget https://github.com/lightvector/KataGo/releases/download/v1.4.5/g170-b30c320x2-s4824661760-d1229536699.bin.gz -O 30b.bin.gz
!wget https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b20c256x2-s5303129600-d1228401921.bin.gz -O 20b.bin.gz
!wget
print('配置环境完毕，已下载当前最强权重：'+strongest_weight)

In [ ]:
# 启动ikatago server
username="2233" # 将双引号中的ikatago修改为自己需要的ikatago用户名
password="6666" # 将双引号中的kinfkong修改为自己需要的ikatago连接密码
%cd ~/work
!./run.sh {username} {password}

In [ ]:
!lsb_release -a # 获取系统版本等信息
!nvidia-smi # 获取GPU及其利用率等信息
!nvcc -V # 查询CUDA版本等信息
!cat /proc/cpuinfo | grep name | sort | uniq # 查询CPU型号

In [ ]:
# benchmark，前提是在运行完第一个单元格后进行
model='28b' # 默认可填18b、28b、40b、30b、20b
visits=5000 # 每个position用这么多计算量
%cd ~/work/data
!./katago benchmark -model {model}.bin.gz -config default_gtp.cfg -v {visits}